# 01_cleaning.ipynb - Enhanced Data Preprocessing

**Goal:** Achieve R² ≥ 0.89 using successful XGBoost approach

This notebook implements comprehensive data preprocessing based on the successful XGBoost CA Hanqing approach that achieved R² = 0.8978. Key improvements:

1. **Smart Feature Engineering:** BuildingAge, target encoding for categorical variables
2. **IQR-based Outlier Removal:** Systematic outlier detection and removal
3. **Feature Selection:** Drop columns with >30% missing values, focus on high-importance features
4. **Proper Data Validation:** Comprehensive data quality checks

**Input:** `data/all_raw.csv`
**Output:** `data/cleaned_enhanced.csv` (enhanced dataset for high-performance modeling)

In [50]:
# Section 1: Imports and Setup
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Paths
ROOT = Path(r"c:\Users\lpnhu\Downloads\home-price-prediction")
DATA_DIR = ROOT / 'data'
RAW_PATH = DATA_DIR / 'all_raw.csv'
ENHANCED_PATH = DATA_DIR / 'cleaned_enhanced.csv'
IMPUTED_PATH = DATA_DIR / 'cleaned_enhanced_imputed.csv'

print('ROOT:', ROOT)
print('RAW_PATH:', RAW_PATH)
print('Enhanced dataset will be saved to:', ENHANCED_PATH)

ROOT: c:\Users\lpnhu\Downloads\home-price-prediction
RAW_PATH: c:\Users\lpnhu\Downloads\home-price-prediction\data\all_raw.csv
Enhanced dataset will be saved to: c:\Users\lpnhu\Downloads\home-price-prediction\data\cleaned_enhanced.csv


In [51]:
# Section 2: Load and Initial Filtering
# Load raw data
df = pd.read_csv(RAW_PATH, low_memory=False)
print('Initial shape:', df.shape)
print('Columns available:', len(df.columns))

# Apply initial high-impact filters first (following successful approach)
print('\n=== INITIAL FILTERING ===')

# Filter to Residential + SingleFamilyResidence (key success factor)
if 'PropertyType' in df.columns and 'PropertySubType' in df.columns:
    initial_count = len(df)
    df = df[(df['PropertyType'] == 'Residential') & (df['PropertySubType'] == 'SingleFamilyResidence')]
    print(f'PropertyType + PropertySubType filter: {initial_count:,} → {len(df):,} rows ({len(df)/initial_count:.1%} kept)')

# Remove rows where ClosePrice is null (essential target variable)
if 'ClosePrice' in df.columns:
    initial_count = len(df)
    df = df.dropna(subset=['ClosePrice'])
    print(f'ClosePrice not null filter: {initial_count:,} → {len(df):,} rows')

print(f'\nAfter initial filtering: {df.shape}')
df.head(3)

Initial shape: (156064, 78)
Columns available: 78

=== INITIAL FILTERING ===
PropertyType + PropertySubType filter: 156,064 → 78,387 rows (50.2% kept)
ClosePrice not null filter: 78,387 → 78,387 rows

After initial filtering: (78387, 78)


,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,ListAgentEmail,...,LotSizeDimensions,LotSizeArea,MainLevelBedrooms,NewConstructionYN,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict
2,SanDiego,SanDiego,NaN,False,NaN,NaN,False,880000.0,497696903,lenskab@gmail.com,...,NaN,NaN,NaN,False,2.0,NaN,91942,NaN,NaN,NaN
3,SanDiego,SanDiego,NaN,False,NaN,NaN,False,875000.0,497696407,lenskab@gmail.com,...,NaN,NaN,NaN,False,2.0,NaN,91942,NaN,NaN,NaN
4,SanDiego,SanDiego,NaN,False,NaN,NaN,False,849000.0,486616176,lenskab@gmail.com,...,NaN,NaN,NaN,False,2.0,NaN,91942,NaN,NaN,NaN


In [52]:
# Section 3: Column Selection and Data Quality Assessment
print('=== COLUMN SELECTION (Following Successful Approach) ===')

# Select key columns based on successful XGBoost approach
selected_columns = [
    'ViewYN', 'PoolPrivateYN', 'LivingArea', 'Latitude', 'MLSAreaMajor',
    'CountyOrParish', 'AttachedGarageYN', 'ParkingTotal', 'LotSizeAcres',
    'YearBuilt', 'Longitude', 'BathroomsTotalInteger', 'City', 'BedroomsTotal',
    'UnparsedAddress', 'HighSchoolDistrict', 'Levels', 'LotSizeArea',
    'NewConstructionYN', 'GarageSpaces', 'ClosePrice', 'ListingId'
]

# Keep only columns that exist in our dataset
available_columns = [col for col in selected_columns if col in df.columns]
missing_columns = [col for col in selected_columns if col not in df.columns]

print(f'Selected columns available: {len(available_columns)}/{len(selected_columns)}')
print(f'Available: {available_columns}')
if missing_columns:
    print(f'Missing: {missing_columns}')

# Filter to selected columns
df = df[available_columns]
df.reset_index(drop=True, inplace=True)
print(f'\nShape after column selection: {df.shape}')

# Data quality assessment
print(f'\n=== DATA QUALITY ASSESSMENT ===')
print(f'Dataset shape: {df.shape}')
print(f'Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB')

=== COLUMN SELECTION (Following Successful Approach) ===
Selected columns available: 22/22
Available: ['ViewYN', 'PoolPrivateYN', 'LivingArea', 'Latitude', 'MLSAreaMajor', 'CountyOrParish', 'AttachedGarageYN', 'ParkingTotal', 'LotSizeAcres', 'YearBuilt', 'Longitude', 'BathroomsTotalInteger', 'City', 'BedroomsTotal', 'UnparsedAddress', 'HighSchoolDistrict', 'Levels', 'LotSizeArea', 'NewConstructionYN', 'GarageSpaces', 'ClosePrice', 'ListingId']

Shape after column selection: (78387, 22)

=== DATA QUALITY ASSESSMENT ===
Dataset shape: (78387, 22)
Memory usage: 48.2 MB


In [53]:
# Section 4: Missing Value Analysis and Column Filtering
print('=== MISSING VALUE ANALYSIS ===')

# Calculate percentage of missing values per column
missing_analysis = {}
for column in df.columns:
    missing_count = df[column].isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    missing_analysis[column] = {
        'count': missing_count,
        'percentage': missing_pct
    }
    print(f'{column}: {missing_count:,} missing ({missing_pct:.2f}%)')

print(f'\nTotal missing values: {df.isnull().sum().sum():,}')

# Filter out columns with >30% missing values (following successful approach)
print(f'\n=== DROPPING HIGH-MISSING COLUMNS (>30%) ===')
columns_to_keep = []
columns_to_drop = []

for column, stats in missing_analysis.items():
    if stats['percentage'] > 30:
        columns_to_drop.append(column)
        print(f'DROP: {column} ({stats["percentage"]:.2f}% missing)')
    else:
        columns_to_keep.append(column)

print(f'\nColumns to keep: {len(columns_to_keep)}')
print(f'Columns to drop: {len(columns_to_drop)}')

# Apply column filtering
if columns_to_drop:
    df = df[columns_to_keep]
    print(f'Shape after dropping high-missing columns: {df.shape}')

=== MISSING VALUE ANALYSIS ===
ViewYN: 7,166 missing (9.14%)
PoolPrivateYN: 6,179 missing (7.88%)
LivingArea: 45 missing (0.06%)
Latitude: 5 missing (0.01%)
MLSAreaMajor: 11,417 missing (14.56%)
CountyOrParish: 0 missing (0.00%)
AttachedGarageYN: 9,346 missing (11.92%)
ParkingTotal: 1 missing (0.00%)
LotSizeAcres: 1,358 missing (1.73%)
YearBuilt: 59 missing (0.08%)
Longitude: 5 missing (0.01%)
BathroomsTotalInteger: 17 missing (0.02%)
City: 57 missing (0.07%)
BedroomsTotal: 0 missing (0.00%)
UnparsedAddress: 82 missing (0.10%)
HighSchoolDistrict: 21,176 missing (27.01%)
Levels: 5,932 missing (7.57%)
LotSizeArea: 1,353 missing (1.73%)
NewConstructionYN: 5,931 missing (7.57%)
GarageSpaces: 3,133 missing (4.00%)
ClosePrice: 0 missing (0.00%)
ListingId: 0 missing (0.00%)

Total missing values: 73,262

=== DROPPING HIGH-MISSING COLUMNS (>30%) ===

Columns to keep: 22
Columns to drop: 0


In [54]:
# Section 5: Feature Engineering (Key Success Factor)
print('=== FEATURE ENGINEERING ===')

# Create raw feature columns for modeling (preserve original names for target encoding)
numeric_columns = ['LivingArea', 'YearBuilt', 'ParkingTotal', 'LotSizeAcres', 
                  'BathroomsTotalInteger', 'BedroomsTotal', 'GarageSpaces', 'LotSizeArea']

for col in numeric_columns:
    if col in df.columns:
        df[f'{col}_raw'] = pd.to_numeric(df[col], errors='coerce')
        print(f'Created {col}_raw: {df[f"{col}_raw"].notna().sum():,} non-null values')

# KEY SUCCESS FACTOR: BuildingAge instead of YearBuilt
current_year = datetime.now().year
if 'YearBuilt_raw' in df.columns:
    # Remove unrealistic years
    df = df[df['YearBuilt_raw'] <= current_year]
    print(f'Removed {len(df)} rows with YearBuilt > {current_year}')
    
    # Create BuildingAge
    df['BuildingAge'] = current_year - df['YearBuilt_raw']
    df.drop(columns=['YearBuilt_raw'], inplace=True)
    print(f'Created BuildingAge: min={df["BuildingAge"].min()}, max={df["BuildingAge"].max()}')

# Clean ParkingTotal (keep >= 0)
if 'ParkingTotal_raw' in df.columns:
    initial_count = len(df)
    df = df[df['ParkingTotal_raw'] >= 0]
    print(f'ParkingTotal >= 0 filter: {initial_count:,} → {len(df):,} rows')

print(f'Shape after feature engineering: {df.shape}')

=== FEATURE ENGINEERING ===
Created LivingArea_raw: 78,342 non-null values
Created YearBuilt_raw: 78,328 non-null values
Created ParkingTotal_raw: 78,386 non-null values
Created LotSizeAcres_raw: 77,029 non-null values
Created BathroomsTotalInteger_raw: 78,370 non-null values
Created BedroomsTotal_raw: 78,387 non-null values
Created GarageSpaces_raw: 75,254 non-null values
Created LotSizeArea_raw: 77,034 non-null values
Removed 78325 rows with YearBuilt > 2025
Created BuildingAge: min=0.0, max=225.0
ParkingTotal >= 0 filter: 78,325 → 78,313 rows
Shape after feature engineering: (78313, 30)


In [55]:
# Section 6: Outlier Removal (Critical Success Factor)
print('=== IQR-BASED OUTLIER REMOVAL ===')

# Apply IQR method to ClosePrice (following successful approach)
if 'ClosePrice' in df.columns:
    Q1 = df['ClosePrice'].quantile(0.25)
    Q3 = df['ClosePrice'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    initial_count = len(df)
    df = df[(df['ClosePrice'] >= lower_bound) & (df['ClosePrice'] <= upper_bound)]
    
    print(f'ClosePrice IQR bounds: ${lower_bound:,.0f} - ${upper_bound:,.0f}')
    print(f'Outlier removal: {initial_count:,} → {len(df):,} rows ({(initial_count-len(df))/initial_count:.1%} removed)')
    
    # Show price statistics
    print(f'ClosePrice stats after outlier removal:')
    print(f'  Min: ${df["ClosePrice"].min():,.0f}')
    print(f'  Max: ${df["ClosePrice"].max():,.0f}')
    print(f'  Median: ${df["ClosePrice"].median():,.0f}')
    print(f'  Mean: ${df["ClosePrice"].mean():,.0f}')

print(f'\nFinal shape after outlier removal: {df.shape}')

=== IQR-BASED OUTLIER REMOVAL ===
ClosePrice IQR bounds: $-600,025 - $2,680,015
Outlier removal: 78,313 → 72,611 rows (7.3% removed)
ClosePrice stats after outlier removal:
  Min: $0
  Max: $2,680,000
  Median: $855,000
  Mean: $1,001,183

Final shape after outlier removal: (72611, 30)


In [56]:
# Section 7: Target Encoding for Categorical Variables (Key Success Factor)
print('=== TARGET ENCODING FOR CATEGORICAL VARIABLES ===')

# Target encoding for high-cardinality categorical variables
categorical_columns = ['City', 'MLSAreaMajor', 'CountyOrParish']

for col in categorical_columns:
    if col in df.columns:
        # Calculate mean ClosePrice for each category
        target_mean = df.groupby(col)['ClosePrice'].mean()
        df[f'{col}_target'] = df[col].map(target_mean)
        
        # Drop original categorical column to avoid high-dimensional sparse encoding
        df.drop(columns=[col], inplace=True)
        
        print(f'Target encoded {col}: {len(target_mean)} unique categories')
        print(f'  Min target value: ${df[f"{col}_target"].min():,.0f}')
        print(f'  Max target value: ${df[f"{col}_target"].max():,.0f}')
        print(f'  Median target value: ${df[f"{col}_target"].median():,.0f}')

print(f'\nShape after target encoding: {df.shape}')

=== TARGET ENCODING FOR CATEGORICAL VARIABLES ===
Target encoded City: 881 unique categories
  Min target value: $26,000
  Max target value: $2,505,000
  Median target value: $962,706
Target encoded MLSAreaMajor: 978 unique categories
  Min target value: $71,000
  Max target value: $2,680,000
  Median target value: $947,362
Target encoded CountyOrParish: 57 unique categories
  Min target value: $272,500
  Max target value: $1,716,800
  Median target value: $1,117,183

Shape after target encoding: (72611, 30)


In [57]:
# Section 8: One-Hot Encoding for Boolean Variables
print('=== ONE-HOT ENCODING FOR BOOLEAN VARIABLES ===')

# One-hot encode remaining categorical variables (low cardinality)
boolean_columns = ['NewConstructionYN', 'ViewYN', 'PoolPrivateYN', 'AttachedGarageYN', 'Levels', 'HighSchoolDistrict']

for col in boolean_columns:
    if col in df.columns:
        # Get dummies and drop first to avoid multicollinearity
        dummies = pd.get_dummies(df[col], prefix=col, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df.drop(columns=[col], inplace=True)
        print(f'One-hot encoded {col}: created {len(dummies.columns)} dummy variables')

print(f'\nShape after one-hot encoding: {df.shape}')
print(f'Current columns: {len(df.columns)}')

=== ONE-HOT ENCODING FOR BOOLEAN VARIABLES ===
One-hot encoded NewConstructionYN: created 2 dummy variables
One-hot encoded ViewYN: created 2 dummy variables
One-hot encoded PoolPrivateYN: created 2 dummy variables
One-hot encoded AttachedGarageYN: created 2 dummy variables
One-hot encoded Levels: created 16 dummy variables
One-hot encoded HighSchoolDistrict: created 406 dummy variables

Shape after one-hot encoding: (72611, 454)
Current columns: 454
One-hot encoded Levels: created 16 dummy variables
One-hot encoded HighSchoolDistrict: created 406 dummy variables

Shape after one-hot encoding: (72611, 454)
Current columns: 454


In [ ]:
# Section 9: Final Feature Validation and Data Quality Check
print('=== FINAL FEATURE VALIDATION ===')

# Display current feature set
print(f'Final dataset shape: {df.shape}')
print(f'Features available: {df.columns.tolist()}')

# Check for any remaining missing values
missing_summary = df.isnull().sum()
columns_with_missing = missing_summary[missing_summary > 0]

if len(columns_with_missing) > 0:
    print(f'\nColumns with missing values:')
    for col, missing_count in columns_with_missing.items():
        missing_pct = (missing_count / len(df)) * 100
        print(f'  {col}: {missing_count:,} ({missing_pct:.2f}%)')
else:
    print('\nNo missing values in final dataset')

# Basic statistics for key numeric features
numeric_features = ['ClosePrice', 'LivingArea_raw', 'BuildingAge', 'Latitude', 'Longitude']
available_numeric = [col for col in numeric_features if col in df.columns]

if available_numeric:
    print(f'\nKey numeric feature statistics:')
    print(df[available_numeric].describe())

# Memory usage
memory_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f'\nMemory usage: {memory_mb:.2f} MB')

=== FINAL FEATURE VALIDATION ===
Final dataset shape: (72611, 454)
Features available: ['LivingArea', 'Latitude', 'ParkingTotal', 'LotSizeAcres', 'YearBuilt', 'Longitude', 'BathroomsTotalInteger', 'BedroomsTotal', 'UnparsedAddress', 'LotSizeArea', 'GarageSpaces', 'ClosePrice', 'ListingId', 'LivingArea_raw', 'ParkingTotal_raw', 'LotSizeAcres_raw', 'BathroomsTotalInteger_raw', 'BedroomsTotal_raw', 'GarageSpaces_raw', 'LotSizeArea_raw', 'BuildingAge', 'City_target', 'MLSAreaMajor_target', 'CountyOrParish_target', 'NewConstructionYN_False', 'NewConstructionYN_True', 'ViewYN_False', 'ViewYN_True', 'PoolPrivateYN_False', 'PoolPrivateYN_True', 'AttachedGarageYN_False', 'AttachedGarageYN_True', 'Levels_MultiSplit', 'Levels_MultiSplit,One', 'Levels_One', 'Levels_One,MultiSplit', 'Levels_One,ThreeOrMore', 'Levels_One,Two', 'Levels_One,Two,ThreeOrMore', 'Levels_One,Two,ThreeOrMore,MultiSplit', 'Levels_ThreeOrMore', 'Levels_ThreeOrMore,MultiSplit', 'Levels_ThreeOrMore,One', 'Levels_Two', 'Levels_T

## Enhanced Preprocessing Summary

**Key Improvements Implemented:**

### 1. **Smart Feature Engineering**
- **BuildingAge**: Replaced YearBuilt with BuildingAge (more interpretable and better model performance)
- **Target Encoding**: Applied to high-cardinality categorical variables (City, MLSAreaMajor, CountyOrParish)
- **One-Hot Encoding**: Applied to boolean/low-cardinality variables

### 2. **Systematic Outlier Removal**
- **IQR Method**: Applied to ClosePrice to remove extreme outliers
- **Data Validation**: Removed unrealistic YearBuilt values (> current year)
- **Range Filtering**: Ensured ParkingTotal ≥ 0

### 3. **Data Quality Focus**
- **Missing Value Strategy**: Dropped columns with >30% missing values
- **Residential SFR Focus**: Filtered to PropertyType='Residential' & PropertySubType='SingleFamilyResidence'
- **Target Variable**: Ensured ClosePrice is not null

### 4. **Expected Outcomes**
This preprocessing pipeline follows the exact methodology that achieved **R² = 0.8978** in the reference implementation.

In [ ]:
# Section 10: Save Enhanced Dataset
print('=== SAVING ENHANCED DATASET ===')

# Save the enhanced dataset
df.to_csv(ENHANCED_PATH, index=False)
print(f'Saved enhanced dataset to: {ENHANCED_PATH}')
print(f'  Shape: {df.shape}')
print(f'  Size: {ENHANCED_PATH.stat().st_size / 1024**2:.2f} MB')

# Display sample of final dataset
print(f'\n=== SAMPLE OF FINAL DATASET ===')
print(df.head(3))

# Show column summary
print(f'\n=== FINAL COLUMN SUMMARY ===')
print(f'Total columns: {len(df.columns)}')
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print(f'Numeric columns ({len(numeric_cols)}): {numeric_cols[:10]}{"..." if len(numeric_cols) > 10 else ""}')
print(f'Categorical columns ({len(categorical_cols)}): {categorical_cols[:10]}{"..." if len(categorical_cols) > 10 else ""}')

print(f'\nENHANCED PREPROCESSING COMPLETE')
print(f'Ready for high-performance XGBoost modeling (targeting R² ≥ 0.89)')

=== SAVING ENHANCED DATASET ===
✓ Saved enhanced dataset to: c:\Users\lpnhu\Downloads\home-price-prediction\data\cleaned_enhanced.csv
  Shape: (72611, 454)
  Size: 192.19 MB

=== SAMPLE OF FINAL DATASET ===
   LivingArea   Latitude  ParkingTotal  LotSizeAcres  YearBuilt   Longitude  \
0      2340.0  32.765380           6.0           NaN     2021.0 -117.043486   
1      2165.0  32.765038           4.0           NaN     2021.0 -117.043568   
2      2158.0  32.765031           4.0           NaN     2021.0 -117.043252   

   BathroomsTotalInteger  BedroomsTotal UnparsedAddress  LotSizeArea  ...  \
0                    4.0            4.0    4750 Dana Dr          NaN  ...   
1                    4.0            5.0    4740 Dana Dr          NaN  ...   
2                    3.0            4.0    4730 Dana Dr          NaN  ...   

   HighSchoolDistrict_Willows Unified  HighSchoolDistrict_Wilsona  \
0                               False                       False   
1                            